In [ ]:
from datetime import datetime
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

from utils import (
    get_mesh_files,
    load_data,
    advect_nodes,
    remesh,
    optimize_mesh,
    compute_mapping,
    get_area_ratio,
    resample_velocities
)

np.seterr(divide='ignore', invalid='ignore')
%matplotlib inline

In [ ]:
mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
mask = np.load(mesh_init_file)['mask']
mask_f = gaussian_filter(mask.astype(float), 1, truncate=1)

In [ ]:
force = False
min_edge_length = 13
max_edge_length = 38
min_edge_angle = 15
min_triangles = 5
search_dist = 50
max_dist0 = 50
min_area = 20
max_mask_fix_node=0.15
max_mask_zero_speed=0.6

sid_dir = '/Data/sim/data/OSISAF_ice_drift_CDR_postproc'
lag_dir = '/data2/Anton/sia/cdr_1991_2023/mesh'
ifiles = sorted(glob.glob(f'{sid_dir}/*/*npz'))[12460:]

idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]
print(len(ifiles), idates[0], idates[-1])

In [ ]:
# MOVE MESH
#for ifile, idate in tqdm(zip(ifiles, idates), total=len(ifiles)):
ifile = ifiles[0]
idate = idates[0]


mesh_src_file, mesh_dst_file = get_mesh_files(idate, lag_dir, mesh_init_file)
#if os.path.exists(mesh_dst_file) and not force:
#    continue
u, v, tri0 = load_data(ifile, mesh_src_file)
u0, v0, fixed_nodes_idx = resample_velocities(u, v, mask_f, xc, yc, tri0.x, tri0.y, max_mask_fix_node, max_mask_zero_speed)
tri_a = advect_nodes(tri0, u0, v0, max_dist0)
tri_r = remesh(tri_a, fixed_nodes_idx, min_area, min_edge_length, min_edge_angle, max_edge_length)
tri_o = optimize_mesh(tri_a, tri_r, fixed_nodes_idx)
src2dst, weights = compute_mapping(tri_a, tri_o, search_dist)
area_ratio = get_area_ratio(tri0, tri_a, tri_o, src2dst, weights)

#print(f'Save mesh file to {mesh_dst_file}')
#np.savez(mesh_dst_file, x=x7, y=y7, t=t7, src2dst=src2dst, weights=weights, ar=area_ratio7)


In [ ]:
plt.tripcolor(tri_o, area_ratio, cmap='coolwarm', vmin=0.8, vmax=1.2)